## Importing Clean Data

In [33]:
#### Importing library
import pandas as pd
import numpy as np
import plotly.express as px
import nbformat
import plotly.graph_objects as go

#### Importing Clean file

In [34]:
orders = pd.read_csv("orders_clean.csv")
products = pd.read_csv("products_clean.csv")
reviews = pd.read_csv("reviews_clean.csv")
users = pd.read_csv("users_clean.csv")

## Creating Product Profiling Table

In [35]:
# Editing initials product 

#Correcting Reviews Rating
# Compute average rating for each product
average_ratings = reviews.groupby('product_id')['rating'].mean().reset_index()
# Compute average review length
average_reviews_length = reviews.groupby('product_id')['review_length'].mean().reset_index()
# Compute amount of reviews per product
review_counts = reviews.groupby('product_id')['id'].count().reset_index()
review_counts.rename(columns={'id': 'review_count'}, inplace=True)
average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)


#Merging table
productstemp = products.merge(average_ratings, left_on="id", right_on="product_id", how= 'left')
productstemp2 = productstemp.merge(average_reviews_length, left_on="id", right_on="product_id", how= 'left')
productstemp2 = productstemp2.drop(["product_id_x","product_id_y"], axis= 1)
productstemp3 = productstemp2.merge(review_counts,left_on="id", right_on="product_id", how= 'left')
productstemp3 = productstemp3.drop(["product_id","rating"],axis=1)
productstemp3

product_profile = productstemp3.copy()
product_profile

,id,category,price,title,vendor,average_rating,review_length,review_count
0,1,Gizmo,29.4633,Rustic Paper Wallet,"Swaniawski, Casper and Hilll",4.625000,172.750000,8.0
1,2,Doohickey,70.0799,Small Marble Shoes,Balistreri-Ankunding,NaN,NaN,NaN
2,3,Doohickey,35.3887,Synergistic Granite Chair,"Murray, Watsica and Wunsch",4.000000,171.000000,7.0
3,4,Doohickey,73.9918,Enormous Aluminum Shirt,Regan Bradtke and Sons,3.000000,167.600000,5.0
4,5,Gadget,82.7451,Enormous Marble Wallet,"Price, Schultz and Daniel",4.000000,146.750000,4.0
...,...,...,...,...,...,...,...,...
195,196,Widget,46.7641,Heavy-Duty Linen Toucan,Balistreri-Muller,NaN,NaN,NaN
196,197,Gizmo,46.7641,Aerodynamic Concrete Lamp,Erika Volkman Group,4.666667,162.833333,6.0
197,198,Gizmo,46.7641,Enormous Copper Shirt,"Considine, Schamberger and Schiller",4.142857,197.000000,7.0
198,199,Widget,76.9533,Mediocre Leather Coat,"Gulgowski, Grimes and Mayer",3.666667,185.333333,6.0


#### Loading Products Profile Table

In [36]:
product_profile.to_csv("product_profile.csv",index= False)

## Creating Users Profiling Table

In [37]:
#Dropping unneccesary information
userstemp = users.drop(["zip", "city","birth_date"],axis= 1)
userstemp

,id,name,email,state,latitude,longitude,source,dayduration,age
0,1,Hudson Borer,borer-hudson@yahoo.com,NE,40.7132,-98.5260,Twitter,2684,38
1,2,Domenica Williamson,williamson-domenica@yahoo.com,IA,41.5813,-92.6991,Affiliate,2500,57
2,3,Lina Heaney,lina.heaney@yahoo.com,MN,46.1197,-92.8416,Facebook,2786,63
3,4,Arnold Adams,adams.arnold@gmail.com,CO,37.9203,-104.9730,Google,2182,32
4,5,Dominique Leffler,leffler.dominique@hotmail.com,NY,42.3490,-77.0567,Twitter,2716,50
...,...,...,...,...,...,...,...,...,...
2495,2496,Jettie Yost,jettie-yost@gmail.com,CO,39.9009,-102.6890,Affiliate,2269,36
2496,2497,Sadye Gibson,sadye.gibson@gmail.com,DE,38.6579,-75.1427,Organic,2959,38
2497,2498,Verner Hamill,verner.hamill@gmail.com,UT,38.3279,-113.0120,Affiliate,3086,62
2498,2499,Cloyd Beer,cloyd-beer@gmail.com,WA,47.7267,-117.1940,Twitter,2881,60


#### Creating total spent from each category table

In [38]:
#Creating total spent from each category table
result = orders.groupby(["user_id", "category"])["total"].sum().reset_index()
# Convert the result to wide format
wide_result = result.pivot(index="user_id", columns="category", values="total").fillna(0)

# Rename columns to indicate total spent
wide_result.columns = [f"total_spent_{col}" for col in wide_result.columns]

# Reset index to make user_id a column again
totalspent = wide_result.reset_index()

# Display result
totalspent

,user_id,total_spent_Doohickey,total_spent_Gadget,total_spent_Gizmo,total_spent_Widget
0,1,189.5193,389.5355,221.8629,1719.2326
1,3,896.4755,126.9100,695.0698,510.8554
2,4,149.8910,0.0000,214.7897,150.5928
3,5,0.0000,0.0000,332.2080,0.0000
4,6,215.3479,350.4764,0.0000,50.1976
...,...,...,...,...,...
1741,2496,115.6694,1288.9272,720.3583,0.0000
1742,2497,441.8439,514.7469,518.7900,0.0000
1743,2498,39.7362,607.4135,218.3368,377.0575
1744,2499,508.7424,543.9881,407.9344,2138.7636


#### Creating Discount Usage Table

In [39]:
# Count total orders per user
total_orders = orders.groupby("user_id")["id_x"].count().reset_index()
total_orders.columns = ["user_id", "total_orders"]

# Count the number of times each user used a discount
discount_usage = orders[orders["discount"] > 0].groupby("user_id")["discount"].count().reset_index()
discount_usage.columns = ["user_id", "discount_usage_count"]

# Merge both dataframes
discount_proportion = total_orders.merge(discount_usage, on="user_id", how="left")

# Fill NaN values (users who never used a discount) with 0
discount_proportion["discount_usage_count"] = discount_proportion["discount_usage_count"].fillna(0)

# Calculate the proportion of orders where a discount was used
discount_proportion["discount_usage_proportion"] = discount_proportion["discount_usage_count"] / discount_proportion["total_orders"]

# Display result
discount_usage = discount_proportion.drop("discount_usage_count", axis=1)
discount_usage

,user_id,total_orders,discount_usage_proportion
0,1,11,0.272727
1,3,10,0.200000
2,4,4,0.250000
3,5,1,0.000000
4,6,4,0.250000
...,...,...,...
1741,2496,12,0.083333
1742,2497,8,0.000000
1743,2498,5,0.200000
1744,2499,17,0.058824


In [42]:
#Merging category spent table with users
userstemp2 = userstemp.merge(totalspent, left_on="id", right_on="user_id", how= "left")
userstemp2 = userstemp2.drop("user_id",axis= 1)

#Merging discount usage table with users
userstemp3 = userstemp2.merge(discount_usage, left_on="id", right_on="user_id", how= "left")
userstemp3

#Creating Email Provider columns
# Extract email provider (domain) without ".com"
userstemp3["email_provider"] = userstemp3["email"].str.split("@").str[1].str.replace(".com", "", regex=False)
userstemp4 = userstemp3.drop(["email","user_id"], axis= 1)
userstemp4

users_profile = userstemp4.copy()



### Loading Users Profile Table

In [43]:
users_profile.to_csv("users_profile.csv")